## 파이썬으로 웹 크롤러 만들기 2일차

## ch.3 크롤링 시작하기
- 여러 페이지, 사이트를 이동하면서 데이터를 수집하는 스크레이퍼 만들기
- 재귀적으로 작업을 수행하는 크롤러를 만들기
    - URL 페이지를 가져오고 -> 해당 페이지 내 다른 URL을 찾고 -> 다시 그 페이지를 가져오고 의 반복
    

### 위키백과를 이용하여 스크레이핑 기초 닦아보기
- 아래의 모든 내용들은 영문 위키백과 사이트를 이용하여 작업을 수행함

In [2]:
#어떤 인물의 페이지 내 내용들 중 다른 인물과 연계되는 URL을 찾아 최단으로 가는 방법 찾아보기
#Kevin_Bacon 의 페이지 내용을 찾아오는 스크레이핑 하기

from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen("https://en.wikipedia.org/wiki/Kevin_Bacon")
bs = BeautifulSoup(html, 'html.parser')

#페이지 내 링크들이 있는 태그와 내용들을 찾아오기 

for link in bs.findAll('a'): #a인 태그를 모두 찾고
    if 'href' in link.attrs: #a 태그 내 href 이름을 가진 속성의 내용을 모두 출력 
        print(link.attrs['href'])
    #조건에 해당하는 경우를 모두 출력했지만 https://wikimediafoundation.org/ 같은 필요없는 url까지 수집됨

/wiki/Wikipedia:Protection_policy#semi
#mw-head
#searchInput
/wiki/Kevin_Bacon_(disambiguation)
/wiki/File:Kevin_Bacon_SDCC_2014.jpg
/wiki/Philadelphia,_Pennsylvania
/wiki/Kyra_Sedgwick
/wiki/Sosie_Bacon
#cite_note-1
/wiki/Edmund_Bacon_(architect)
/wiki/Michael_Bacon_(musician)
/wiki/Holly_Near
http://baconbros.com/
#cite_note-2
#cite_note-actor-3
/wiki/Footloose_(1984_film)
/wiki/JFK_(film)
/wiki/A_Few_Good_Men
/wiki/Apollo_13_(film)
/wiki/Mystic_River_(film)
/wiki/Balto_(film)
/wiki/Sleepers
/wiki/The_Woodsman_(2004_film)
/wiki/Animal_House
/wiki/Diner_(1982_film)
/wiki/Tremors_(film)
/wiki/Crazy,_Stupid,_Love
/wiki/Friday_the_13th_(1980_film)
/wiki/Flatliners
/wiki/The_River_Wild
/wiki/Wild_Things_(film)
/wiki/Stir_of_Echoes
/wiki/Hollow_Man
/wiki/Frost/Nixon_(film)
/wiki/X-Men:_First_Class
/wiki/Black_Mass_(film)
/wiki/Patriots_Day_(film)
/wiki/Fox_Broadcasting_Company
/wiki/The_Following
/wiki/HBO
/wiki/Taking_Chance
/wiki/Golden_Globe_Award
/wiki/Screen_Actors_Guild_Award
/wiki/P

### 조건을 더욱 특정해서 필요한 url만 가져오기
- 페이지 내 생애, 논란과 같은 항목으로 이동하는 내용들을 제외하고
- 외부 url을 제외하고
- 오직 위키백과의 다른 내용으로 이동하는 url만 가져오기

<br />

**과정**     
- 먼저 태그가 div이고 속성 id의 변수가 bodyContent인 내용만 찾고
- 그 다음 /wiki/로 시작되는 url만 가져오는데
- 해당 url안에는 콜론이 포함되어있지 않아야 함

In [3]:
import re

html = urlopen("https://en.wikipedia.org/wiki/Kevin_Bacon")
bs = BeautifulSoup(html, 'html.parser')

for link in bs.find('div', {'id' : 'bodyContent'}).findAll('a',
                                                          href = re.compile('^(\/wiki\/)((?!:).)*$')):
    #https://www3.ntu.edu.sg/home/ehchua/programming/howto/Regexe.html 참조
    #여기서 (?!:)는 Negative Lookahead 이며 ?! 다음에 어떤 문자열을 입력하는 형식으로 사용됨
    #Negative Lookahead는 ?!다음에 쓰인 문자를 제외하고 모든 문자를 매칭함
    
    if 'href' in link.attrs:
        print(link.attrs['href'])

/wiki/Kevin_Bacon_(disambiguation)
/wiki/Philadelphia,_Pennsylvania
/wiki/Kyra_Sedgwick
/wiki/Sosie_Bacon
/wiki/Edmund_Bacon_(architect)
/wiki/Michael_Bacon_(musician)
/wiki/Holly_Near
/wiki/Footloose_(1984_film)
/wiki/JFK_(film)
/wiki/A_Few_Good_Men
/wiki/Apollo_13_(film)
/wiki/Mystic_River_(film)
/wiki/Balto_(film)
/wiki/Sleepers
/wiki/The_Woodsman_(2004_film)
/wiki/Animal_House
/wiki/Diner_(1982_film)
/wiki/Tremors_(film)
/wiki/Crazy,_Stupid,_Love
/wiki/Friday_the_13th_(1980_film)
/wiki/Flatliners
/wiki/The_River_Wild
/wiki/Wild_Things_(film)
/wiki/Stir_of_Echoes
/wiki/Hollow_Man
/wiki/Frost/Nixon_(film)
/wiki/Black_Mass_(film)
/wiki/Patriots_Day_(film)
/wiki/Fox_Broadcasting_Company
/wiki/The_Following
/wiki/HBO
/wiki/Taking_Chance
/wiki/Golden_Globe_Award
/wiki/Screen_Actors_Guild_Award
/wiki/Primetime_Emmy_Award
/wiki/I_Love_Dick_(TV_series)
/wiki/Golden_Globe_Award_for_Best_Actor_%E2%80%93_Television_Series_Musical_or_Comedy
/wiki/The_Guardian
/wiki/Academy_Award
/wiki/Hollywood

### 사용자 정의 함수로 작성해 향후 작업을 더욱 간단히 하기

수행할 작업
- getLinks 함수 생성
    - 위키 백과 내 검색하고 싶은 항목의 페이지에 들어가서
    - 해당 페이지와 연결된 다른 위키백과 항목으로 넘어갈 수 있는 모든 url을 긁어오는 작업
- getLinks로 긁어온 url중 임의의 url을 선택하는 작업의 반복

In [4]:
import datetime as dt
import random as rd

rd.seed(dt.datetime.now()) #현재 시간을 기준으로 의사 난수를 생성할 시드번호 생성

def getLinks(articleUrl):
    html = urlopen('http://en.wikipedia.org{}'.format(articleUrl))
    bs = BeautifulSoup(html, 'html.parser')
    
    return bs.find('div', {'id' : 'bodyContent'}).findAll('a',href = re.compile('^(\/wiki\/)((?!:).)*$'))

In [5]:
links = getLinks('/wiki/Kevin_Bacon')

In [ ]:
while len(links) > 0:
    newArticle = links[rd.randint(0, len(links) - 1)].attrs['href']
    print(newArticle)
    links = getLinks(newArticle)
    # 매우 많은 iter를 하므로 실행시키지 않음

### 사이트 전체의 url들을 긁고 싶은 경우

- 말 그대로 어떤 사이트에 존재하는 모든 페이지들의 모든 url들을 수집하기 위한 작업
- 핵심은 작업을 재귀적으로 하는 것
- 이미 긁어왔던 page를 다시 이용하거나 긁으면 안되므로 set안에 데이터를 담는 작업을 수행
    - set는 내부 데이터의 중복을 허용하지 않으므로 자연스럽게 유일한 값들만 사용 가능

In [8]:
pages = set()

def getLinks(pageUrl):
    global pages
    html = urlopen('http://en.wikipedia.org{}'.format(pageUrl))
    bs = BeautifulSoup(html, 'html.parser')
    
    for link in bs.findAll('a', href = re.compile(r'^(\/wiki\/)')):
        if 'href' in link.attrs:
            if link.attrs['href'] not in pages: #pages 세트 안에 없는 url일 경우
                newPage = link.attrs['href'] 
                print(newPage) #url을 출력하고
                pages.add(newPage) #pages 세트 안에 값을 넣고
                getLinks(newPage)

In [ ]:
getLinks('')
    #3번째 수집하고 스크레이퍼가 멈춰버림

/wiki/Wikipedia
/wiki/Wikipedia:Protection_policy#semi
/wiki/Wikipedia:Requests_for_page_protection


### 구조화된 데이터 수집

- 페이지에서 제목을 출력하고
- 텍스트 내용 중 일부를 출력하고
- 편집 링크들을 출력하는 작업 수행
    - 위의 작업들을 반복적으로 수행하는 함수 생성

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

pages = set()

def getLinks(pageUrl):
    global pages
    html = urlopen('http://en.wikipedia.org' + pageUrl) #유저가 원하는 위키피디아 페이지를 열고
    bs = BeautifulSoup(html, 'html.parser')
    
    try:
        print(bs.h1.get_text()) #해당 페이지의 제목을 가져오고
        print(bs.find(id = 'mw-content-text').findAll('p')[0]) #해당 페이지 내용의 일부를 가져오고
        print(bs.find(id = 'ca-edit').find('span').find('a').attrs['href']) #편집 링크를 보여준다
        
    except AttributeError:
        print('This page is missing something! No worries though!')
        
    for link in bs.findAll('a', href = re.compile('^(\/wiki\/)')):
        if 'href' in link.attrs:
            if link.attrs['href'] not in pages:
                newPage = link.attrs['href']
                print('--------------------------\n' + newPage)
                pages.add(newPage)
                getLinks(newPage)

In [2]:
getLinks('')

Main Page
<p><b><a href="/wiki/Dali_(goddess)" title="Dali (goddess)">Dali</a></b> is a <a href="/wiki/List_of_hunting_deities" title="List of hunting deities">hunting goddess</a> from <a href="/wiki/Georgian_mythology" title="Georgian mythology">the mythology</a> of the <a href="/wiki/Georgians" title="Georgians">Georgian people</a>. The patron of hoofed wild mountain animals, she was said to reward hunters who obeyed her <a href="/wiki/Taboo" title="Taboo">taboos</a> and to punish violators. She was usually described as a beautiful nude woman with golden hair and glowing skin, although she sometimes <a href="/wiki/Shapeshifting" title="Shapeshifting">took on the form</a> of her favored animals. Stories depict her taking human lovers and jealously killing them, and later clashing with her rival <a href="/wiki/Tetri_Giorgi" title="Tetri Giorgi">Saint George</a>. After the rise of <a href="/wiki/Christianity_in_Georgia_(country)" title="Christianity in Georgia (country)">Christianity in

Help:What links here
<p class="mw-empty-elt">
</p>
This page is missing something! No worries though!
--------------------------
/wiki/Wikipedia:Project_namespace#How-to_and_information_pages
Wikipedia:Project namespace
<p class="mw-empty-elt">
</p>
This page is missing something! No worries though!
--------------------------
/wiki/Wikipedia:Policies_and_guidelines
Wikipedia:Policies and guidelines
<p>Wikipedia's <b>policies and guidelines</b> are developed by the community to describe best practices, clarify principles, resolve conflicts, and otherwise further our goal of creating a free, reliable encyclopedia. There is no need to read any policy or guideline pages to start editing. The <a href="/wiki/Wikipedia:Five_pillars" title="Wikipedia:Five pillars">five pillars</a> are a popular summary of the most pertinent principles.
</p>
This page is missing something! No worries though!
--------------------------
/wiki/Wikipedia:WikiProject_Politics
Wikipedia:WikiProject Politics
<p>Welcom

Statistical area (United States)
<p><span style="font-size:110%;"><a href="/wiki/List_of_United_States_cities_by_population" title="List of United States cities by population">Population</a></span>
</p>
This page is missing something! No worries though!
--------------------------
/wiki/File:Pano_Manhattan2007_amk.jpg
File:Pano Manhattan2007 amk.jpg
<p><a class="internal" href="//upload.wikimedia.org/wikipedia/commons/8/8d/Pano_Manhattan2007_amk.jpg" title="Pano Manhattan2007 amk.jpg">Original file</a> ‎<span class="fileInfo">(2,764 × 800 pixels, file size: 1.61 MB, MIME type: <span class="mime-type">image/jpeg</span>)</span>
</p>
This page is missing something! No worries though!
--------------------------
/wiki/User:AngMoKio
User:AngMoKio
<p><b>If you want to contact me than please do that on my Commons-Account <a class="external text" href="https://commons.wikimedia.org/wiki/User:AngMoKio">(AngMoKio)</a></b><br/>
<b>My old account was <a class="mw-redirect" href="/wiki/User:AngMo" ti

KeyboardInterrupt: 

In [3]:
pages

{'/wiki/Des_Moines,_Iowa',
 '/wiki/Des_Moines_(disambiguation)',
 '/wiki/Des_Moines_metropolitan_area',
 '/wiki/File:A_coloured_voting_box.svg',
 '/wiki/File:Essay.png',
 '/wiki/File:Essay.svg',
 '/wiki/File:Pano_Manhattan2007_amk.jpg',
 '/wiki/File:People_icon.svg',
 '/wiki/File:People_icon_dead.svg',
 '/wiki/File:Question_book-new.svg',
 '/wiki/File:To_Commons.svg',
 '/wiki/Help:What_links_here',
 '/wiki/List_of_ISO_639-1_codes',
 '/wiki/List_of_metropolitan_statistical_areas',
 '/wiki/List_of_primary_statistical_areas_of_the_United_States',
 '/wiki/Metropolitan_statistical_area',
 '/wiki/SVG',
 '/wiki/SVG_(disambiguation)',
 '/wiki/Scalable_Vector_Graphics',
 '/wiki/Special:WhatLinksHere/File:People_icon.svg',
 '/wiki/Special:WhatLinksHere/File:To_Commons.svg',
 '/wiki/User:AngMoKio',
 '/wiki/User:RjLesch',
 '/wiki/User:StevenDH',
 '/wiki/User_talk:Robert_Merkel',
 '/wiki/User_talk:Robert_Merkel/ancient',
 '/wiki/Wikipedia',
 '/wiki/Wikipedia:Babel',
 '/wiki/Wikipedia:Graphic_Lab/Re